In [1]:
pip install tensorflow-addons

In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


In [8]:
num_classes = 10
input_shape = (32, 32, 3)

AUTO=tf.data.AUTOTUNE
learning_rate = 0.15
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 50
dropout_rate = 0.5
temperature = 0.1



# Load the train and test data splits
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Display shapes of train and test datasets
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

test_dataset=(tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .shuffle(1024)
    .batch(batch_size)
    .prefetch(AUTO)
)




x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [9]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
        layers.RandomWidth(0.2),
        layers.RandomHeight(0.2),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)


In [10]:
def create_encoder(backbone=keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )):
    

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = backbone(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model


encoder = create_encoder()
encoder.summary()



Model: "cifar10-encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 sequential_1 (Sequential)   (None, 32, 32, 3)         7         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,807
Trainable params: 23,519,360
Non-trainable params: 45,447
_________________________________________________________________


In [15]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable
    learning_rate=0.001
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model


In [12]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="cifar-encoder_with_projection-head"
    )
    return model


## model definition with the api

In [13]:

def apply_resnet_block(x,downsample,conv_by_block):
    
    depth_input=x.shape[-1]
    
    
    if downsample:
        depth=depth_input*2
        skiped=layers.Conv2D(depth,1,strides=(2,2),activation=None)(x)#linear projection
        x=layers.Conv2D(depth,3,strides=(2,2), activation='relu',padding="same")(x)
        x=layers.BatchNormalization()(x)
    else:
        depth=depth_input
        skiped=x
        x=layers.Conv2D(depth,3, activation='relu',padding="same")(x)
        x=layers.BatchNormalization()(x)
        
    for i in range(1,conv_by_block-1):
        x=layers.Conv2D(depth,3, activation='relu',padding="same")(x)
        x=layers.BatchNormalization()(x)
    
    x=layers.Conv2D(depth,3,padding="same")(x)#don't apply activation to the last 
         
    x=skiped+x
    x=layers.ReLU()(x)
    x=layers.BatchNormalization()(x)
   
    return x

def get_resnet_backbone(input_shape,hparams):
    """
    """
    assert hparams["conv_by_block"]>=2
    inputs=layers.Input((32,32,3))
    x=layers.Conv2D(hparams["depth_first_convolution"],7,strides=(2,2),activation='relu',padding="same")(inputs)
    x=layers.BatchNormalization()(x)
    for block in range(1,hparams["number_of_block"]+1):
        x=apply_resnet_block(x,block in hparams["downsample_num"],hparams["conv_by_block"])
       
        
    x=layers.Conv2D(hparams["output_dim"],3, activation='relu',padding="same")(x)
    x=layers.BatchNormalization()(x)
    if hparams["globalPoolingType"]=="Mean":
        
        x=layers.GlobalAveragePooling2D()(x)
 
    return tf.keras.Model(inputs,x)



hparams_resnet={"depth_first_convolution":64,
                "output_dim":2048,
                "number_of_block":5,
                "downsample_num":[4,5],
                "conv_by_block":2,
                "globalPoolingType":"Mean"
            }




resnet=get_resnet_backbone(input_shape,hparams_resnet)

resnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_14 (Conv2D)             (None, 16, 16, 64)   9472        ['input_6[0][0]']                
                                                                                                  
 batch_normalization_12 (BatchN  (None, 16, 16, 64)  256         ['conv2d_14[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_15 (Conv2D)             (None, 16, 16, 64)   36928       ['batch_normalization_12[0]

                                                                                                  
 re_lu_9 (ReLU)                 (None, 4, 4, 256)    0           ['tf.__operators__.add_9[0][0]'] 
                                                                                                  
 batch_normalization_22 (BatchN  (None, 4, 4, 256)   1024        ['re_lu_9[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 conv2d_27 (Conv2D)             (None, 4, 4, 2048)   4720640     ['batch_normalization_22[0][0]'] 
                                                                                                  
 batch_normalization_23 (BatchN  (None, 4, 4, 2048)  8192        ['conv2d_27[0][0]']              
 ormalization)                                                                                    
          

In [16]:
early_stop_unsup=tf.keras.callbacks.EarlyStopping(
    monitor="loss", patience=3, restore_best_weights=True
)
learning_rate=0.15
resnet=get_resnet_backbone(input_shape,hparams_resnet)

encoder = create_encoder(resnet)

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.SGD(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs,callbacks=[early_stop_unsup]
)

Model: "cifar-encoder_with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 cifar10-encoder (Functional  (None, 2048)             6112775   
 )                                                               
                                                                 
 dense_2 (Dense)             (None, 128)               262272    
                                                                 
Total params: 6,375,047
Trainable params: 6,368,512
Non-trainable params: 6,535
_________________________________________________________________
Epoch 1/50
189/189 [==============================] - 78s 371ms/step - loss: 5.6447
Epoch 2/50
189/189 [==============================] - 37s 194ms/step - loss: 5.1392
Epoch 3/50
189/189 [============================

KeyboardInterrupt: 

In [17]:
early_stop=tf.keras.callbacks.EarlyStopping(
    monitor="sparse_categorical_accuracy", patience=3, restore_best_weights=True
)


classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=x_train, y=y_train,validation_data=test_dataset, batch_size=batch_size, epochs=1,callbacks=[early_stop])

history = classifier.fit(x=x_train, y=y_train,validation_data=test_dataset, batch_size=batch_size, epochs=num_epochs,callbacks=[early_stop])


accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")


189/189 [==============================] - 27s 118ms/step - loss: 1.0604 - sparse_categorical_accuracy: 0.7680 - val_loss: 0.7616 - val_sparse_categorical_accuracy: 0.7742
Epoch 1/50
189/189 [==============================] - 14s 72ms/step - loss: 0.9576 - sparse_categorical_accuracy: 0.7875 - val_loss: 0.7580 - val_sparse_categorical_accuracy: 0.7830
Epoch 2/50
189/189 [==============================] - 13s 70ms/step - loss: 0.9377 - sparse_categorical_accuracy: 0.7933 - val_loss: 0.7870 - val_sparse_categorical_accuracy: 0.7851
Epoch 3/50
189/189 [==============================] - 13s 68ms/step - loss: 0.9276 - sparse_categorical_accuracy: 0.7956 - val_loss: 0.7571 - val_sparse_categorical_accuracy: 0.7788
Epoch 4/50
189/189 [==============================] - 12s 65ms/step - loss: 0.7678 - sparse_categorical_accuracy: 0.7959 - val_loss: 0.6552 - val_sparse_categorical_accuracy: 0.7865
Epoch 5/50
189/189 [==============================] - 13s 66ms/step - loss: 0.7987 - sparse_categori

KeyboardInterrupt: 

In [18]:

history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=35,callbacks=[early_stop_unsup]
)

Epoch 1/35
189/189 [==============================] - 32s 168ms/step - loss: 4.2224
Epoch 2/35
189/189 [==============================] - 31s 164ms/step - loss: 4.1910
Epoch 3/35
189/189 [==============================] - 33s 174ms/step - loss: 4.1569
Epoch 4/35
189/189 [==============================] - 31s 164ms/step - loss: 4.1242
Epoch 5/35
189/189 [==============================] - 31s 165ms/step - loss: 4.0968
Epoch 6/35
189/189 [==============================] - 31s 164ms/step - loss: 4.0681
Epoch 7/35
189/189 [==============================] - 32s 170ms/step - loss: 4.0334
Epoch 8/35
189/189 [==============================] - 32s 171ms/step - loss: 4.0090
Epoch 9/35
189/189 [==============================] - 32s 168ms/step - loss: 3.9928
Epoch 10/35
189/189 [==============================] - 32s 169ms/step - loss: 3.9676
Epoch 11/35
189/189 [==============================] - 32s 168ms/step - loss: 3.9514
Epoch 12/35
189/189 [==============================] - 31s 165ms/step - lo

In [19]:
early_stop=tf.keras.callbacks.EarlyStopping(
    monitor="sparse_categorical_accuracy", patience=3, restore_best_weights=True
)


classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=x_train, y=y_train,validation_data=test_dataset, batch_size=batch_size, epochs=1,callbacks=[early_stop])

history = classifier.fit(x=x_train, y=y_train,validation_data=test_dataset, batch_size=batch_size, epochs=num_epochs,callbacks=[early_stop])


accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

189/189 [==============================] - 16s 69ms/step - loss: 0.2983 - sparse_categorical_accuracy: 0.9026 - val_loss: 0.5352 - val_sparse_categorical_accuracy: 0.8524
Epoch 1/50
189/189 [==============================] - 13s 67ms/step - loss: 0.2182 - sparse_categorical_accuracy: 0.9295 - val_loss: 0.5280 - val_sparse_categorical_accuracy: 0.8594
Epoch 2/50
189/189 [==============================] - 12s 65ms/step - loss: 0.2070 - sparse_categorical_accuracy: 0.9333 - val_loss: 0.5335 - val_sparse_categorical_accuracy: 0.8620
Epoch 3/50
189/189 [==============================] - 13s 67ms/step - loss: 0.2018 - sparse_categorical_accuracy: 0.9338 - val_loss: 0.5599 - val_sparse_categorical_accuracy: 0.8566
Epoch 4/50
189/189 [==============================] - 12s 65ms/step - loss: 0.1937 - sparse_categorical_accuracy: 0.9354 - val_loss: 0.5412 - val_sparse_categorical_accuracy: 0.8597
Epoch 5/50
189/189 [==============================] - 12s 66ms/step - loss: 0.1976 - sparse_categoric

KeyboardInterrupt: 

In [21]:
accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

313/313 [==============================] - 4s 11ms/step - loss: 0.5480 - sparse_categorical_accuracy: 0.8600
Test accuracy: 86.0%
